# Sentiment Analysis on Go Emotions with Transformers

## Setup & Preprocessing

### Install Packages

In [1]:
# %pip install pandas numpy torch nltk datasets transformers scikit-learn

### NLTK Setup

In [2]:
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure necessary resources are available
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alber\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alber\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alber\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Preprocessing Functions

In [3]:
def preprocess_text(text):
    """Clean, normalize, tokenize, remove stopwords, and lemmatize text."""
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'\S*@\S*\s?', '', text)  # Remove emails
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    
    tokens = word_tokenize(text)  # Tokenization
    
    # Remove stopwords and lemmatize
    processed_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return ' '.join(processed_tokens)

### Feature Extraction

In [4]:
def extract_features(text):
    """Extract additional features from text"""
    features = {}
    original_text = text
    
    features['text_length'] = len(text)
    features['word_count'] = len(text.split())
    features['has_question'] = 1 if '?' in original_text else 0
    features['has_exclamation'] = 1 if '!' in original_text else 0
    features['has_emoticon'] = 1 if any(emoticon in original_text.lower() for emoticon in [':)', ':(', ':D', ';)', ':P', 'XD']) else 0
    
    # All caps words ratio (potential indicator of intensity)
    words = text.split()
    if words:
        features['caps_ratio'] = sum(1 for word in words if word.isupper()) / len(words)
    else:
        features['caps_ratio'] = 0
    
    return features

## Data Preparation

### Load Dataset

In [5]:
import pandas as pd
from datasets import load_dataset

# Loading the dataset
data = load_dataset('go_emotions')
df_train = pd.DataFrame(data["train"])
df_val = pd.DataFrame(data["validation"])
df_test = pd.DataFrame(data["test"])

# Display dataset info
print(f"Training examples: {len(df_train)}")
print(f"Validation examples: {len(df_val)}")
print(f"Test examples: {len(df_test)}")

# Check the columns to understand what we're working with
print(f"Columns: {df_train.columns}")

# Get list of emotion labels
emotion_labels = data['train'].features['labels'].feature.names
num_labels = len(emotion_labels)
print(f"Number of emotion labels: {num_labels}")
print(f"Emotion labels: {emotion_labels}")

Training examples: 43410
Validation examples: 5426
Test examples: 5427
Columns: Index(['text', 'labels', 'id'], dtype='object')
Number of emotion labels: 28
Emotion labels: ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


### Apply Preprocessing

In [6]:
# Add preprocessed text columns
print("Preprocessing text data...")
df_train['cleaned_text'] = df_train['text'].apply(preprocess_text)
df_val['cleaned_text'] = df_val['text'].apply(preprocess_text)
df_test['cleaned_text'] = df_test['text'].apply(preprocess_text)

# Extract features
print("Extracting text features...")
train_features = df_train['text'].apply(extract_features).apply(pd.Series)
val_features = df_val['text'].apply(extract_features).apply(pd.Series)
test_features = df_test['text'].apply(extract_features).apply(pd.Series)

# Add features to dataframes
for feature in train_features.columns:
    df_train[feature] = train_features[feature]
    df_val[feature] = val_features[feature]
    df_test[feature] = test_features[feature]

# Display sample
print("\nSample data with features:")
print(df_train[['text', 'cleaned_text', 'text_length', 'word_count', 'has_question', 'has_exclamation', 'caps_ratio']].head())

Preprocessing text data...
Extracting text features...

Sample data with features:
                                                text  \
0  My favourite food is anything I didn't have to...   
1  Now if he does off himself, everyone will thin...   
2                     WHY THE FUCK IS BAYLESS ISOING   
3                        To make her feel threatened   
4                             Dirty Southern Wankers   

                                        cleaned_text  text_length  word_count  \
0                 favourite food anything didnt cook         59.0        11.0   
1  everyone think he laugh screwing people instea...        112.0        20.0   
2                                fuck bayless isoing         30.0         6.0   
3                               make feel threatened         27.0         5.0   
4                              dirty southern wanker         22.0         3.0   

   has_question  has_exclamation  caps_ratio  
0           0.0              0.0    0.090909  

### Dataset Class

In [7]:
import torch

# Add cleaned text to the dataset
data["train"] = data["train"].add_column("cleaned_text", df_train["cleaned_text"].tolist())
data["validation"] = data["validation"].add_column("cleaned_text", df_val["cleaned_text"].tolist())
data["test"] = data["test"].add_column("cleaned_text", df_test["cleaned_text"].tolist())

# Create a custom dataset class that incorporates text features
class GoEmotionsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, features):
        self.encodings = encodings
        self.labels = labels
        self.features = features
    
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        
        # Add text features
        for feature_name, feature_values in self.features.items():
            item[feature_name] = torch.tensor(feature_values[idx], dtype=torch.float)
        
        return item
    
    def __len__(self):
        return len(self.labels)

### Initialize Tokenizer

In [8]:
from transformers import AutoTokenizer

# Initialize tokenizer
model_name = "distilbert-base-uncased"  # A good balance of performance and speed
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the datasets
def tokenize_function(examples):
    return tokenizer(
        examples["cleaned_text"] if "cleaned_text" in examples else examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

# Prepare datasets for training
train_dataset = data["train"].map(tokenize_function, batched=True)
val_dataset = data["validation"].map(tokenize_function, batched=True)
test_dataset = data["test"].map(tokenize_function, batched=True)

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

### Extract Features

In [9]:
# Extract numeric features from dataframes
train_numeric_features = {
    'text_length': df_train['text_length'].values,
    'word_count': df_train['word_count'].values,
    'has_question': df_train['has_question'].values,
    'has_exclamation': df_train['has_exclamation'].values,
    'has_emoticon': df_train['has_emoticon'].values,
    'caps_ratio': df_train['caps_ratio'].values
}

val_numeric_features = {
    'text_length': df_val['text_length'].values,
    'word_count': df_val['word_count'].values,
    'has_question': df_val['has_question'].values,
    'has_exclamation': df_val['has_exclamation'].values,
    'has_emoticon': df_val['has_emoticon'].values,
    'caps_ratio': df_val['caps_ratio'].values
}

test_numeric_features = {
    'text_length': df_test['text_length'].values,
    'word_count': df_test['word_count'].values,
    'has_question': df_test['has_question'].values,
    'has_exclamation': df_test['has_exclamation'].values,
    'has_emoticon': df_test['has_emoticon'].values,
    'caps_ratio': df_test['caps_ratio'].values
}


### Process Labels

In [10]:
import numpy as np

# Convert labels to the expected format for multi-label classification
def process_labels(examples):
    # Convert label indices to multi-hot encoding
    labels = np.zeros((len(examples["labels"]), num_labels))
    for i, label_list in enumerate(examples["labels"]):
        labels[i, label_list] = 1
    return {"labels": labels.tolist()}

train_dataset = train_dataset.map(process_labels, batched=True)
val_dataset = val_dataset.map(process_labels, batched=True)
test_dataset = test_dataset.map(process_labels, batched=True)

# Create custom datasets with features
# Get processed labels
train_labels = np.array(train_dataset["labels"])
val_labels = np.array(val_dataset["labels"])
test_labels = np.array(test_dataset["labels"])

# Remove labels from encodings
train_encodings = {k: v for k, v in train_dataset.to_dict().items() if k != "labels"}
val_encodings = {k: v for k, v in val_dataset.to_dict().items() if k != "labels"}
test_encodings = {k: v for k, v in test_dataset.to_dict().items() if k != "labels"}

# Create custom datasets
custom_train_dataset = GoEmotionsDataset(train_encodings, train_labels, train_numeric_features)
custom_val_dataset = GoEmotionsDataset(val_encodings, val_labels, val_numeric_features)
custom_test_dataset = GoEmotionsDataset(test_encodings, test_labels, test_numeric_features)

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

## Model Building

### Model Architecture

In [11]:
# Create a custom model that incorporates text features
from transformers import DistilBertPreTrainedModel, DistilBertModel
from torch import nn

class EmotionClassifier(DistilBertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.distilbert = DistilBertModel(config)
        
        # Number of additional features
        self.num_features = 6  # text_length, word_count, etc.
        
        # Add a feature extractor network
        self.feature_extractor = nn.Sequential(
            nn.Linear(self.num_features, 32),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        # Combine BERT embeddings with our features
        self.classifier = nn.Sequential(
            nn.Linear(config.hidden_size + 32, config.hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(config.hidden_size, config.num_labels)
        )
        
        # Initialize weights
        self.post_init()
        
    def forward(self, input_ids=None, attention_mask=None, head_mask=None, 
                inputs_embeds=None, labels=None, output_attentions=None, 
                output_hidden_states=None, return_dict=None, **kwargs):
        
        # Get text features from kwargs
        text_features = []
        for feature_name in ['text_length', 'word_count', 'has_question', 
                            'has_exclamation', 'has_emoticon', 'caps_ratio']:
            if feature_name in kwargs:
                text_features.append(kwargs[feature_name].unsqueeze(1))
        
        # Concatenate all features
        if text_features:
            text_features = torch.cat(text_features, dim=1)
        else:
            # If no features provided, use zeros as fallback
            text_features = torch.zeros(input_ids.shape[0], self.num_features, device=input_ids.device)
        
        # Get BERT embeddings
        outputs = self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict
        )
        
        # Get the CLS token embedding
        bert_output = outputs[0][:, 0]  # CLS token
        
        # Process the features
        feature_output = self.feature_extractor(text_features)
        
        # Concatenate BERT output with features
        combined = torch.cat([bert_output, feature_output], dim=1)
        
        # Final classification
        logits = self.classifier(combined)
        
        # loss = None
        # if labels is not None:
        #     loss_fct = nn.BCEWithLogitsLoss()
        #     loss = loss_fct(logits.view(-1, self.num_labels), 
        #                    labels.float().view(-1, self.num_labels))
        
        loss = None
        if labels is not None:
            # Use class weights if available
            if hasattr(self, 'pos_weights'):
                loss_fct = nn.BCEWithLogitsLoss(pos_weight=self.pos_weights)
            else:
                loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), 
                        labels.float().view(-1, self.num_labels))

        return (loss, logits) if loss is not None else logits

### Initialize Model

In [12]:
# Load pre-trained DistilBERT config and modify it
from transformers import AutoConfig
config = AutoConfig.from_pretrained(
    model_name,
    num_labels=num_labels,
    problem_type="multi_label_classification"
)

# Initialize our custom model
model = EmotionClassifier.from_pretrained(
    model_name,
    config=config
)

# Check if GPU is available and move model to GPU
if torch.cuda.is_available():
    model = model.to('cuda')
    print("Model loaded on GPU")
else:
    print("Model loaded on CPU")


Some weights of EmotionClassifier were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.0.bias', 'classifier.0.weight', 'classifier.3.bias', 'classifier.3.weight', 'feature_extractor.0.bias', 'feature_extractor.0.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded on GPU


In [13]:
# Compute class weights based on label distribution
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Reshape labels for weight calculation (train_labels is multi-hot encoded)
flat_labels = np.where(train_labels == 1)[1]  # Get indices of positive labels

# Compute class weights
class_weights = compute_class_weight('balanced', 
                                    classes=np.unique(flat_labels), 
                                    y=flat_labels)
                                    
# Convert to tensor and move to appropriate device
pos_weights = torch.tensor(class_weights, dtype=torch.float)
if torch.cuda.is_available():
    pos_weights = pos_weights.to('cuda')

# Pass these weights to the model
model.pos_weights = pos_weights

### Training Setup

In [14]:
import torch
from sklearn.metrics import (accuracy_score, f1_score, precision_score,
                             recall_score)
from transformers import TrainingArguments


# Define metrics for evaluation
def compute_metrics(pred):
    predictions = (pred.predictions > 0).astype(int)
    labels = pred.label_ids
    
    # Compute metrics
    accuracy = accuracy_score(labels.flatten(), predictions.flatten())
    f1 = f1_score(labels, predictions, average='weighted')
    precision = precision_score(labels, predictions, average='weighted', zero_division=0)
    recall = recall_score(labels, predictions, average='weighted', zero_division=0)
    
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# # Set up training arguments
# training_args = TrainingArguments(
#     output_dir="./results",
#     learning_rate=5e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     push_to_hub=False,
#     fp16=torch.cuda.is_available(),  # Enable mixed precision if GPU available
# )


# Set up training arguments with improvements
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,  # Slightly lower learning rate
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,  # Increase training epochs
    weight_decay=0.01,
    evaluation_strategy="steps",  # Evaluate more frequently
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # Optimize for F1 score
    greater_is_better=True,
    push_to_hub=False,
    fp16=torch.cuda.is_available(),
    logging_steps=100,
    warmup_steps=500,  # Add warmup steps
    lr_scheduler_type="linear",  # Use linear scheduler
    gradient_accumulation_steps=2,  # Effectively doubles batch size
)

c:\Users\alber\anaconda3\envs\mindscribe\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Training & Evaluation

### Train Model

In [15]:
from transformers import Trainer

# Initialize trainer with our custom datasets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=custom_train_dataset,
    eval_dataset=custom_val_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: albert-negoro89 (albert-negoro89-arizona-state-university). Use `wandb login --relogin` to force relogin


  0%|          | 0/13570 [00:00<?, ?it/s]

{'loss': 0.6703, 'grad_norm': 1.4818263053894043, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.07}
{'loss': 0.4467, 'grad_norm': 1.4410593509674072, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.15}
{'loss': 0.2633, 'grad_norm': 0.895915687084198, 'learning_rate': 1.2e-05, 'epoch': 0.22}
{'loss': 0.1856, 'grad_norm': 0.6379570364952087, 'learning_rate': 1.6000000000000003e-05, 'epoch': 0.29}
{'loss': 0.1727, 'grad_norm': 0.451177716255188, 'learning_rate': 2e-05, 'epoch': 0.37}


  0%|          | 0/340 [00:00<?, ?it/s]

{'eval_loss': 0.176142156124115, 'eval_accuracy': 0.9580064240956242, 'eval_f1': 0.0, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_runtime': 3.6661, 'eval_samples_per_second': 1480.056, 'eval_steps_per_second': 92.742, 'epoch': 0.37}
{'loss': 0.1736, 'grad_norm': 1.2803378105163574, 'learning_rate': 1.984697781178271e-05, 'epoch': 0.44}
{'loss': 0.1816, 'grad_norm': 0.5326987504959106, 'learning_rate': 1.969395562356542e-05, 'epoch': 0.52}
{'loss': 0.1839, 'grad_norm': 0.5067772269248962, 'learning_rate': 1.9540933435348126e-05, 'epoch': 0.59}
{'loss': 0.1719, 'grad_norm': 0.4756167531013489, 'learning_rate': 1.9387911247130835e-05, 'epoch': 0.66}
{'loss': 0.1571, 'grad_norm': 0.4957443177700043, 'learning_rate': 1.9234889058913545e-05, 'epoch': 0.74}


...

  0%|          | 0/340 [00:00<?, ?it/s]

{'eval_loss': 0.12615908682346344, 'eval_accuracy': 0.9632589121162656, 'eval_f1': 0.37310288232502486, 'eval_precision': 0.640266833217612, 'eval_recall': 0.325705329153605, 'eval_runtime': 3.2342, 'eval_samples_per_second': 1677.71, 'eval_steps_per_second': 105.127, 'epoch': 9.58}
{'loss': 0.0516, 'grad_norm': 1.1355153322219849, 'learning_rate': 7.253251721499618e-07, 'epoch': 9.65}
{'loss': 0.0508, 'grad_norm': 2.1274962425231934, 'learning_rate': 5.723029839326702e-07, 'epoch': 9.73}
{'loss': 0.0527, 'grad_norm': 1.015350341796875, 'learning_rate': 4.192807957153788e-07, 'epoch': 9.8}
{'loss': 0.0528, 'grad_norm': 1.2331136465072632, 'learning_rate': 2.662586074980873e-07, 'epoch': 9.87}
{'loss': 0.0512, 'grad_norm': 1.1303362846374512, 'learning_rate': 1.1323641928079573e-07, 'epoch': 9.95}


  0%|          | 0/340 [00:00<?, ?it/s]

{'eval_loss': 0.1265140324831009, 'eval_accuracy': 0.9634695382023064, 'eval_f1': 0.3760206364743733, 'eval_precision': 0.6413025386459763, 'eval_recall': 0.32601880877742945, 'eval_runtime': 3.0425, 'eval_samples_per_second': 1783.38, 'eval_steps_per_second': 111.749, 'epoch': 9.95}
{'train_runtime': 1015.1144, 'train_samples_per_second': 427.637, 'train_steps_per_second': 13.368, 'train_loss': 0.09082458361862508, 'epoch': 10.0}


TrainOutput(global_step=13570, training_loss=0.09082458361862508, metrics={'train_runtime': 1015.1144, 'train_samples_per_second': 427.637, 'train_steps_per_second': 13.368, 'total_flos': 1.43909582570496e+16, 'train_loss': 0.09082458361862508, 'epoch': 10.0})

### Evaluate Model

In [16]:
# Evaluate on test set
test_results = trainer.evaluate(custom_test_dataset)
print(f"Test results: {test_results}")

  0%|          | 0/340 [00:00<?, ?it/s]

Test results: {'eval_loss': 0.1243186742067337, 'eval_accuracy': 0.9636868567216826, 'eval_f1': 0.37498611678788357, 'eval_precision': 0.6347143582701688, 'eval_recall': 0.3220097961763312, 'eval_runtime': 3.3762, 'eval_samples_per_second': 1607.427, 'eval_steps_per_second': 100.705, 'epoch': 10.0}
